In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 10
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
queue3 = Queue(model,'Queue3',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(2.0))
queue3.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))

In [5]:
# Set routing matrix with serial queue2->queue3 routing
P = model.init_routing_matrix()

P.set(jobclass1, jobclass1, delay, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, queue3, 1.0)  # Serial routing: queue2 -> queue3
P.set(jobclass1, jobclass1, queue3, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 1.9092 1.9092 1.9998  1.9998 0.9436 0.9436
 Queue1   class1 6.0179 0.9444 6.4394  6.4394 0.9300 0.9300
 Queue2   class1 0.8792 0.4812 0.9201  0.9201 0.9439 0.9439
 Queue3   class1 5.0972 0.9244 5.5832  5.5832 0.9432 0.9432
   Join   class1 4.2477 0.9547 2.2373  2.2373 0.9354 0.9354
SOLVER 2: MVA
Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1  1.6935 1.6935 2.0000  2.0000 0.8468 0.8468
 Queue1   class1  5.5256 0.8468 6.5256  6.5256 0.8468 0.8468
 Queue2   class1  0.7342 0.4234 0.8671  0.8671 0.8468 0.8468
 Queue3   class1  5.5256 0.8468 6.5256  6.5256 0.8468 0.8468
   Join   class1 13.1340 0.0000 5.1703  5.1703 0.8468 0.8468
